In [2]:
!pip install keras4torch
!pip install einops

import os
import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,MyRegreLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

^C
^C


In [ ]:
##Load DAta
X_test=pd.DataFrame(np.load('Data/year/N_test.npy')).astype(np.float32)
y_test=pd.DataFrame(np.load('Data/year/y_test.npy')).astype(np.float32)

X_train=pd.DataFrame(np.load('Data/year/N_train.npy')).astype(np.float32)
Y_train=pd.DataFrame(np.load('Data/year/y_train.npy')).astype(np.float32)


X_valid=pd.DataFrame(np.load('Data/year/N_val.npy')).astype(np.float32)
y_valid=pd.DataFrame(np.load('Data/year/y_val.npy')).astype(np.float32)

In [ ]:
#Normalise In
scalerX = StandardScaler()

scalerX.fit(X_train+np.random.normal(0,0.000001,size=X_train.shape))

X_train = scalerX.transform(X_train).astype(np.float32)
X_valid = scalerX.transform(X_valid).astype(np.float32)
X_test  = scalerX.transform(X_test).astype(np.float32)


In [ ]:
scale_factor=Y_train.std()
y_test  =(y_test-Y_train.mean())/scale_factor
y_valid =(y_valid-Y_train.mean())/scale_factor
Y_train =(Y_train-Y_train.mean())/scale_factor

In [ ]:
checkpoint='saved/savefileYE'

In [ ]:
def True_MSE(p,t):
    return np.mean((scale_factor.values[0]*(t- p))  **2)

In [ ]:
Or_model = MainModel(
    categories        = (),
    num_continuous    = 90,
    dim               = 128,
    dim_out           = 1,
    depth             = 6,
    heads             = 8,
    attn_dropout      = 0.25 ,
    ff_dropout        = 0.25,
    U                 = 2,
    cases             = 16

)
no_model = Num_Cat(Or_model,num_number=90)
model    = keras4torch.Model(no_model,).build([90])
no_model.reset_Sample_size(8)

#Warm UP
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.001, total_iters=1,  verbose=False)
model.compile(optimizer=optimizer, loss=MyRegreLoss(0.1,1), metrics=['mse'])
callbacks=[ModelCheckpoint(checkpoint,monitor='val_mse',mode='min'),LRScheduler(sch)]
model.fit(X_train, Y_train.values,
                      epochs=5, batch_size=512,
                      validation_data=(X_valid,y_valid.values),
                      verbose=2,validation_batch_size=128,
                      callbacks=callbacks)
#Main Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyRegreLoss(0.01,1), metrics=['mse'])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, factor=0.5,min_lr=0.00001))
callbacks=[ModelCheckpoint('savefile6YE',monitor='val_mse',mode='min'),scheduler]
model.fit(X_train, Y_train.values,
        epochs=30, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=128,
        callbacks=callbacks)


model.load_weights(checkpoint)

Out2=pd.DataFrame()

for i in range(0,64):

        Out2[i]=model.predict(X_test,batch_size=128)[:,0]
Test = (True_MSE(Out2.mean(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))


Train on 370972 samples, validate on 92743 samples:
Epoch 1/5 - 595s - loss: 1.1750 - mse: 1.2508 - val_loss: 0.9901 - val_mse: 1.0453 - lr: 1e-07
Epoch 2/5 - 595s - loss: 0.7779 - mse: 0.8095 - val_loss: 0.7177 - val_mse: 0.7427 - lr: 1e-04
Epoch 3/5 - 592s - loss: 0.7100 - mse: 0.7341 - val_loss: 0.6955 - val_mse: 0.7181 - lr: 1e-04
Epoch 4/5 - 594s - loss: 0.6892 - mse: 0.7112 - val_loss: 0.6815 - val_mse: 0.7027 - lr: 1e-04
Epoch 5/5 - 596s - loss: 0.6767 - mse: 0.6974 - val_loss: 0.6848 - val_mse: 0.7065 - lr: 1e-04
Train on 370972 samples, validate on 92743 samples:
Epoch 1/40 - 593s - loss: 0.7095 - mse: 0.7117 - val_loss: 0.6754 - val_mse: 0.6772 - lr: 1e-03
Epoch 2/40 - 594s - loss: 0.6734 - mse: 0.6750 - val_loss: 0.6869 - val_mse: 0.6886 - lr: 1e-03
Epoch 3/40 - 595s - loss: 0.6633 - mse: 0.6648 - val_loss: 0.6612 - val_mse: 0.6626 - lr: 1e-03
Epoch 4/40 - 594s - loss: 0.6561 - mse: 0.6575 - val_loss: 0.6532 - val_mse: 0.6545 - lr: 1e-03
Epoch 5/40 - 592s - loss: 0.6526 - ms

In [ ]:
model.load_weights('savefile6YE')

no_model.reset_Sample_size(1)
Out2=pd.DataFrame()

for i in range(0,64):

        Out2[i]=model.predict(X_test,batch_size=4096)[:,0]
Test = (True_MSE(Out2.mean(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))


In [ ]:
(True_MSE(Out2.mean(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))


75.83779